# Yacht UX Helper 사용 설명서

Jupyter에서 **입력 편의 + 상태 자동관리**를 제공하는 헬퍼입니다.

기존 엔진(`yacht_dp_opt_fastpath.py`) 위에 올려 쓰세요.

### 설치/준비

In [ ]:
import os, yacht_dp_opt_fastpath as y
import gzip, pickle

SLIM = os.path.join(os.getcwd(),"yacht_dp_cache_slim.pkl.gz")

with gzip.open(SLIM, "rb") as f:
    d = pickle.load(f)
y.Q_MEMO.update(d.get("Q", {}))
y.EV0_VEC.update(d.get("EV0", {}))
print("loaded slim -> Q:", len(y.Q_MEMO), "EV0:", len(y.EV0_VEC))

## 한 줄 추천 함수: `choose_best`

```python
choose_best(dice, remaining=None, upper_sum=0, r=None)
```

- `dice`: 주사위 입력(아래 모두 허용)
  - 정수: `11456`
  - 문자열: `"11456"`
  - 리스트/튜플: `[1,1,4,5,6]`
- `remaining`(남은 카테고리):
  - `None` 또는 `"all"` → 전부 남음
  - 혼용 입력 OK: `["A","Y","FH"]`, `["Aces","Yacht"]`, `[0, 11]` 등
- `upper_sum`: 상단 누계(0~63, 캡 자동)
- `r`:
  - `None` → **이번 턴 “어느 칸에 적을지”** 추천 (final)
  - `1` 또는 `2` → **지금 “무엇을 들고 갈지”** 추천 (hold)

### 예시

```python
# 들고 가기 추천 (리롤 2회 남음)
choose_best(11456, remaining="all", upper_sum=0, r=2)

# 이번 턴에 적을 칸 추천
choose_best("34556", remaining=["C","4K","LS"], upper_sum=40)  # r=None 기본
```

### 카테고리 약어 (입력 간소화)

| 약어  | 정식명        |
|-----| ------------- |
| `1` | Aces          |
| `2` | Twos          |
| `3` | Threes        |
| `4` | Fours         |
| `5` | Fives         |
| `6` | Sixes         |
| `C` | Choice        |
| `K` | FourKind      |
| `H` | FullHouse     |
| `S` | SmallStraight |
| `L` | LargeStraight |
| `Y` | Yacht         |

`remaining`에 약어/정식명/인덱스를 **섞어서** 넣어도 됩니다. 예: `["1","Choice",11]`


## 게임 상태 자동 관리: `YachtGame`

게임 중 **남은 칸/상단합/기록**을 자동으로 관리합니다.

```python
g = YachtGame()          # 새 게임
g.status()               # 현재 점수판/남은 칸/상단합

# 1) 들고 갈 추천 (r=1/2)
g.recommend_hold(44116, r=2)

# 2) 이번 턴에 적을 칸 추천
g.recommend_category(44116)

# 3) 실제로 기록하기
g.take(44116)            # 최적 칸 자동 선택해서 기록
g.take(22256, "2")       # 특정 칸(Twos)에 강제로 기록

g.status()               # 진행 상황 확인
```

### 메서드 요약

- `g.recommend_hold(dice, r)`
   → `{"keep_hist", "keep_example", "reroll_count", "expected_value"}`
- `g.recommend_category(dice)`
   → `{"best_category", "total_value", "immediate", "future", "ranked":[...]}`
- `g.take(dice, category=None)`
   → 상태 갱신 + 결과 딕셔너리 반환
   (`category=None`이면 최적 칸 자동 선택, 약어/정식명/인덱스 모두 허용)
- `g.status()`
   → `{"sheet_used", "remaining", "upper_sum", "total_scored"}`
- `g.reset()`
   → 게임 초기화

## 출력 해석(요점)

### `choose_best_hold` / `g.recommend_hold`

- `keep_hist` : (1~6 눈별 **들고 갈 개수**)
- `keep_example` : 사람이 보기 쉬운 예시 리스트
- `reroll_count` : 다시 굴릴 주사위 개수
- `expected_value` : **이대로 진행 시 게임 종료까지의 기대 총점(EV)**

### `choose_best_category_final` / `g.recommend_category`

- `best_category` : 이번 턴 적을 최적 칸
- `immediate` : 지금 적으면 얻는 점수
- `future` : 남은 칸으로 끝까지 최적플레이 기대점(상단 보너스 자동 반영)
- `total_value = immediate + future` : **이번 선택의 EV**
- `ranked` : 모든 후보의 `(total, immediate, future)` 정렬표

> 모든 value/EV는 “점수 기대값” 입니다 (확률 아님).

## 편의 함수

```
roll(n=5)        # n개 주사위 랜덤 굴림 -> [..]
sort_dice(d)     # 주사위 정렬 출력 (입력은 int/str/list 모두 허용)
```

## 오류 메시지 자주 묻는 것

- “dice must be …” → 주사위는 **5자리 int/str** 또는 **길이 5 list/tuple**(각 1~6) 필요.
- “unknown category …” → 약어 표를 참고하거나 정식명/인덱스를 사용.

In [ ]:
# === Yacht UX helper  ===
import random
from collections import Counter

# -------- 입력 헬퍼 --------
SHORT = {
    "1":"Aces","2":"Twos","3":"Threes","4":"Fours","5":"Fives","6":"Sixes",
    "C":"Choice","K":"FourKind","H":"FullHouse","S":"SmallStraight",
    "L":"LargeStraight","Y":"Yacht"
}
NAME2IDX = {c:i for i,c in enumerate(y.CATS)}
IDX2NAME = {i:c for c,i in NAME2IDX.items()}

def parse_dice(d):
    """12345 / '12345' / [1,2,3,4,5] / (..)-> [..]"""
    if isinstance(d, int): s = str(d)
    elif isinstance(d, str): s = d.strip()
    elif isinstance(d, (list, tuple)) and len(d)==5:
        lst = list(d);
        if not all(1<=x<=6 for x in lst): raise ValueError("dice must be 1..6")
        return lst
    else:
        raise ValueError("dice must be int/str of 5 digits or 5-long list/tuple")
    if len(s)!=5 or not s.isdigit(): raise ValueError("dice int/str must be 5 digits like 11456")
    lst = [int(ch) for ch in s]
    if not all(1<=x<=6 for x in lst): raise ValueError("digits must be 1..6")
    return lst

def parse_remaining(rem):
    """
    rem 입력 형태:
      - None / 'all' -> 전체
      - ['Aces','Yacht'] / ['A','Y'] / ['4K','FH'] 등
      - [이름과 섞어서 가능] 예: ['A', 'Choice', 'LS']
      - 인덱스 리스트도 가능: [0, 11]
    """
    if rem is None or (isinstance(rem, str) and rem.lower()=="all"):
        return list(range(12))
    out = []
    for x in rem:
        if isinstance(x, int):
            out.append(x)
        else:
            s = str(x).strip()
            s = SHORT.get(s, s)  # 약어→정식명
            if s not in NAME2IDX: raise ValueError(f"unknown category: {x}")
            out.append(NAME2IDX[s])
    return out

def dice_hist(dice):
    cnt = Counter(dice)
    return tuple(cnt.get(i,0) for i in range(1,7))

# -------- 추천 한 방 래퍼 --------
def choose_best(dice, remaining=None, upper_sum=0, r=None):
    """
    r=None  -> 이번에 '어느 칸에 적을지' 추천 (final)
    r=1/2   -> 지금 무엇을 들고 갈지 추천 (hold)
    dice는 int/'12345'/list 모두 허용
    remaining은 None/'all'/약어/정식명/인덱스 혼용 허용
    """
    dice = parse_dice(dice)
    rem_idx = parse_remaining(remaining if remaining is not None else y.CATS)
    rem_names = [IDX2NAME[i] for i in rem_idx]
    if r is None:
        return y.choose_best_category_final(dice, rem_names, upper_sum)
    else:
        return y.choose_best_hold(dice, rem_names, upper_sum, r)

# -------- 게임 상태 관리 클래스 --------
class YachtGame:
    """
    점수표와 남은 카테고리를 자동 관리.
    - game.recommend_hold(dice, r)
    - game.recommend_category(dice)
    - game.take(dice, category=None)   # category 미지정이면 최적 자동
    - game.status()                    # 현재 점수표/남은 칸/상단합
    - game.reset()
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.remaining = set(range(12))
        self.sheet = {c: None for c in y.CATS}
        self.upper_sum = 0
        self.total_scored = 0
        self.history = []  # (dice, chosen_cat_idx, score)

    @property
    def remaining_names(self):
        return [IDX2NAME[i] for i in sorted(self.remaining)]

    def recommend_hold(self, dice, r):
        dice = parse_dice(dice)
        rem_names = self.remaining_names
        return y.choose_best_hold(dice, rem_names, self.upper_sum, r)

    def recommend_category(self, dice):
        dice = parse_dice(dice)
        rem_names = self.remaining_names
        return y.choose_best_category_final(dice, rem_names, self.upper_sum)

    def _cat_to_idx(self, cat):
        if cat is None: return None
        if isinstance(cat, int):
            if cat not in self.remaining: raise ValueError("category not remaining")
            return cat
        s = SHORT.get(str(cat).strip(), str(cat).strip())
        if s not in NAME2IDX: raise ValueError(f"unknown category: {cat}")
        idx = NAME2IDX[s]
        if idx not in self.remaining: raise ValueError(f"{s} already used")
        return idx

    def take(self, dice, category=None):
        """
        지금 주사위로 카테고리에 기록하고 상태 업데이트.
        category=None이면 최적 자동 선택.
        return: dict(info)
        """
        dice = parse_dice(dice)
        h = dice_hist(dice)

        if category is None:
            rec = self.recommend_category(dice)
            best_name = rec["best_category"]
            cat_idx = NAME2IDX[best_name]
            immediate = rec["immediate"]
            future = rec["future"]
        else:
            cat_idx = self._cat_to_idx(category)
            immediate = y.score_from_hist6(h, cat_idx)
            best_name = IDX2NAME[cat_idx]
            # 미래 가치는 참고용으로만 계산
            up2 = y.cap_up(self.upper_sum + (immediate if cat_idx in y.UPPER_SET else 0))
            up_mask=low_mask=0
            for c in self.remaining:
                if c==cat_idx: continue
                if c<6: up_mask|=(1<<c)
                else:   low_mask|=(1<<(c-6))
            tail = y.upper_bonus(up2) if (up_mask==0 and low_mask==0) else y.Q(up_mask, low_mask, up2)
            future = tail

        # 상태 갱신
        self.sheet[best_name] = immediate
        self.remaining.remove(cat_idx)
        if cat_idx in y.UPPER_SET:
            self.upper_sum = y.cap_up(self.upper_sum + immediate)
        self.total_scored += immediate
        self.history.append((dice, cat_idx, immediate))

        # 게임 종료 시 보너스 자동 반영 정보(참고용)
        done = (len(self.remaining)==0)
        bonus = y.upper_bonus(self.upper_sum) if done else None

        return {
            "chosen": best_name,
            "immediate": immediate,
            "future_if_optimal": future,
            "upper_sum_now": self.upper_sum,
            "total_scored_now": self.total_scored,
            "bonus_at_end_if_any": bonus,
            "remaining": self.remaining_names
        }

    def status(self):
        used = {k:v for k,v in self.sheet.items() if v is not None}
        return {
            "sheet_used": used,
            "remaining": self.remaining_names,
            "upper_sum": self.upper_sum,
            "total_scored": self.total_scored
        }

# -------- 편의 함수: 주사위 굴리기/정렬 보기 --------
def roll(n=5): return [random.randint(1,6) for _ in range(n)]
def sort_dice(d): return sorted(parse_dice(d))